# 🗺️ ERA5 Simple Visualization (Fixed)
## Pick variable and time → plot!

**Domain:** 10-40°N, 30-60°E  
**Period:** April 2024 (hourly data)

In [ ]:
import os
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import ipywidgets as widgets
from IPython.display import display

plt.rcParams.update({"figure.dpi": 100})

## 📂 Load Data
**⚠️ Update `data_dir` if your files are in a different location!**

In [ ]:
# Change this path if needed
data_dir = "/mnt/user-data/uploads"

print("Loading ERA5 data...")

# Load all files
ds_inv = xr.open_dataset(os.path.join(data_dir, "ERA-5_10-40N_30-60E_INVARIANTS.nc"))
ds_uv10 = xr.open_dataset(os.path.join(data_dir, "ERA-5_UV10_10-40N_30-60E_April2024.nc"))
ds_temp = xr.open_dataset(os.path.join(data_dir, "ERA-5_TEMP_10-40N_30-60E_April2024.nc"))
ds_precip = xr.open_dataset(os.path.join(data_dir, "ERA-5_PRECIP_10-40N_30-60E_April2024.nc"))
ds_cloud = xr.open_dataset(os.path.join(data_dir, "ERA-5_CLOUD_10-40N_30-60E_April2024.nc"))
ds_uv200 = xr.open_dataset(os.path.join(data_dir, "ERA-5_UV200_10-40N_30-60E_April2024.nc"))

print("✅ All files loaded!")

# Get time info
times = ds_temp.time.values
time_labels = [str(t)[:16] for t in times]
print(f"📅 {time_labels[0]} to {time_labels[-1]}")
print(f"⏰ {len(times)} hours total")

## 🎨 Setup

In [ ]:
# Helper function for coordinates
def get_coords(ds):
    lat = 'lat' if 'lat' in ds.coords else 'latitude'
    lon = 'lon' if 'lon' in ds.coords else 'longitude'
    return lat, lon

# All available variables
# Format: 'Display Name': (dataset, variable, units, colormap)
all_variables = {
    'Surface Geopotential': (ds_inv, 'var129', 'm²/s²', 'terrain'),
    'Land-Sea Mask': (ds_inv, 'var172', '0-1', 'RdBu_r'),
    
    '10m U-wind': (ds_uv10, 'var165', 'm/s', 'RdBu_r'),
    '10m V-wind': (ds_uv10, 'var166', 'm/s', 'RdBu_r'),
    '10m Wind Speed': (ds_uv10, 'speed', 'm/s', 'plasma'),
    
    '2m Temperature': (ds_temp, 'var167', '°C', 'RdYlBu_r'),
    '2m Dewpoint': (ds_temp, 'var168', '°C', 'RdYlBu_r'),
    'Relative Humidity': (ds_temp, 'rh', '%', 'BrBG'),
    
    'Hourly Precipitation': (ds_precip, 'var228', 'mm', 'YlGnBu'),
    
    'Cloud Base Height': (ds_cloud, 'var23', 'm', 'viridis'),
    'Low Cloud Cover': (ds_cloud, 'var186', '0-1', 'gray_r'),
    'Medium Cloud Cover': (ds_cloud, 'var187', '0-1', 'gray_r'),
    'High Cloud Cover': (ds_cloud, 'var188', '0-1', 'gray_r'),
    'Total Cloud Cover': (ds_cloud, 'var164', '0-1', 'gray_r'),
    
    '200hPa U-wind': (ds_uv200, 'var131', 'm/s', 'RdBu_r'),
    '200hPa V-wind': (ds_uv200, 'var132', 'm/s', 'RdBu_r'),
    '200hPa Wind Speed': (ds_uv200, 'speed', 'm/s', 'plasma'),
}

print(f"✅ {len(all_variables)} variables available")

## 🗺️ Plotting Function (FIXED)

In [ ]:
def plot_map(variable_name, time_index):
    """Plot the selected variable at the selected time"""
    
    ds, var_name, units, cmap = all_variables[variable_name]
    lat_coord, lon_coord = get_coords(ds)
    lat = ds[lat_coord].values
    lon = ds[lon_coord].values
    
    # Flag for time-varying data
    is_time_varying = True
    
    # Get data - handle calculated variables
    if var_name == 'speed':
        # Wind speed calculation
        if '10m' in variable_name:
            u = ds['var165']
            v = ds['var166']
        else:  # 200hPa
            u = ds['var131']
            v = ds['var132']
        
        # Check if time dimension exists and select
        if 'time' in u.dims:
            u = u.isel(time=time_index)
            v = v.isel(time=time_index)
        else:
            is_time_varying = False
        
        data = np.sqrt(u**2 + v**2)
        
    elif var_name == 'rh':
        # Relative humidity calculation
        t2m = ds['var167']
        td2m = ds['var168']
        
        # Check if time dimension exists and select
        if 'time' in t2m.dims:
            t2m = t2m.isel(time=time_index)
            td2m = td2m.isel(time=time_index)
        else:
            is_time_varying = False
        
        # Convert to Celsius
        t2m_c = t2m - 273.15
        td2m_c = td2m - 273.15
        
        # Calculate RH
        data = 100 * np.exp((17.625 * td2m_c) / (243.04 + td2m_c)) / np.exp((17.625 * t2m_c) / (243.04 + t2m_c))
        
    else:
        # Regular variable
        var_data = ds[var_name]
        
        if 'time' in var_data.dims:
            data = var_data.isel(time=time_index)
        else:
            data = var_data  # Time-invariant
            is_time_varying = False
    
    # Unit conversions for regular variables
    if 'Temperature' in variable_name or 'Dewpoint' in variable_name:
        if var_name in ['var167', 'var168']:
            data = data - 273.15  # K to C
    elif 'Precipitation' in variable_name:
        data = data * 1000  # m to mm
    
    # Extract numpy array (handle both DataArray and numpy array)
    if hasattr(data, 'values'):
        data_array = data.values
    else:
        data_array = data
    
    # Create map
    fig = plt.figure(figsize=(12, 8))
    ax = plt.axes(projection=ccrs.PlateCarree())
    
    ax.set_extent([lon.min(), lon.max(), lat.min(), lat.max()], crs=ccrs.PlateCarree())
    ax.add_feature(cfeature.COASTLINE, linewidth=0.5)
    ax.add_feature(cfeature.BORDERS, linewidth=0.3, alpha=0.5)
    ax.add_feature(cfeature.LAND, alpha=0.1)
    ax.gridlines(draw_labels=True, linewidth=0.5, alpha=0.5)
    
    im = ax.pcolormesh(lon, lat, data_array, 
                       cmap=cmap, 
                       transform=ccrs.PlateCarree(),
                       shading='auto')
    
    cbar = plt.colorbar(im, ax=ax, orientation='horizontal', pad=0.05, shrink=0.8)
    cbar.set_label(units, fontsize=10)
    
    # Title
    if is_time_varying:
        time_str = time_labels[time_index]
        ax.set_title(f"{variable_name}\n{time_str}", fontsize=12, fontweight='bold')
    else:
        ax.set_title(f"{variable_name} (Time-invariant)", fontsize=12, fontweight='bold')
    
    plt.tight_layout()
    plt.show()

print("✅ Plotting function ready")

## 🎛️ Interactive Controls
**Select variable and time, then click Plot!**

In [ ]:
# Variable selector
var_dropdown = widgets.Dropdown(
    options=list(all_variables.keys()),
    value='2m Temperature',
    description='Variable:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='450px')
)

# Time slider (hourly)
time_slider = widgets.IntSlider(
    value=0,
    min=0,
    max=len(times)-1,
    step=1,
    description='Hour:',
    style={'description_width': '100px'},
    layout=widgets.Layout(width='700px')
)

# Time label
time_label = widgets.Label(
    value=f"📅 {time_labels[0]}",
    layout=widgets.Layout(width='300px')
)

def update_time_label(change):
    time_label.value = f"📅 {time_labels[change['new']]}"
    
time_slider.observe(update_time_label, names='value')

# Plot button
plot_button = widgets.Button(
    description='📊 Plot Map',
    button_style='success',
    layout=widgets.Layout(width='180px', height='45px'),
    style={'font_weight': 'bold'}
)

output = widgets.Output()

def on_plot_click(b):
    with output:
        output.clear_output(wait=True)
        try:
            plot_map(var_dropdown.value, time_slider.value)
        except Exception as e:
            print(f"❌ Error: {e}")
            import traceback
            traceback.print_exc()

plot_button.on_click(on_plot_click)

# Display
controls = widgets.VBox([
    widgets.HTML("<h2 style='color: #2e7d32;'>🗺️ ERA5 Interactive Visualization</h2>"),
    widgets.HTML("<hr style='border: 1px solid #ccc;'>"),
    widgets.HTML("<h4>Select Variable:</h4>"),
    var_dropdown,
    widgets.HTML("<br><h4>Select Time (Hourly):</h4>"),
    time_slider,
    time_label,
    widgets.HTML("<br>"),
    plot_button,
    widgets.HTML("<hr style='border: 1px solid #ccc;'>")
], layout=widgets.Layout(padding='20px', border='2px solid #2e7d32', border_radius='10px'))

display(controls, output)

print("\n✅ Ready! Select variable and time, then click 'Plot Map'")